<a href="https://colab.research.google.com/github/cesarrivera52/Concentracion2/blob/main/Regresiones%20NO%20Lineales/Dashboard/Dashboard_Act_3_6_REGRESION_NO_LINEAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LIBRERIAS

In [10]:
#Instalamos la libreria STREAMLIT
%pip install streamlit

In [11]:
#Instalamos la libreria PLOTLY
%pip install plotly

In [12]:
#Instalamos la libreria para análisis de variables categóricas
%pip install funpymodeling

In [13]:
#Instalamos el tunnel local (node.js)
#Para instalar npm en VS
#1. Desde Google escribir node.js
#2. Instalar la versión más recomendada
!npm install localtunnel


up to date, audited 23 packages in 559ms

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [14]:
%pip install streamlit folium streamlit-folium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.8/327.8 kB 4.8 MB/s eta 0:00:00


# Codigo Dashboard

In [94]:
%%writefile app.py
#Creamos el archivo de la APP en el interprete principal (Phyton)

#############################IMPLEMENTACIÓN DE DASHBOARD################################

#Verificamos que todas las librerias se puedan importar
#Importamos librerias
import streamlit as st
import plotly.express as px
import pandas as pd
import numpy as np
from funpymodeling.exploratory import freq_tbl
import folium
from streamlit_folium import folium_static
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
import seaborn as sns


#################################################################

#Definimos la instancia de streamlit
@st.cache_resource

#################################################################

#Creamos la función de carga de datos
def load_data():
    #Lectura del archivo csv sin indice
    data=pd.read_csv("rio_de_janeiro_USD.csv")
    data.drop(columns='Unnamed: 0', inplace=True)
    #Se crea una copia del frame original
    data_copia = data.copy()
    #Se detectan los valores unicos de la columna
    np.unique(data_copia['host_is_superhost'])

    #Se reemplazan los datos categoricos a numerico
    data_copia['host_is_superhost'] = data_copia['host_is_superhost'].replace('f',0)
    data_copia['host_is_superhost'] = data_copia['host_is_superhost'].replace('t',1)
    data_copia['host_is_superhost'] = data_copia['host_is_superhost'].astype(int)

    #Se detectan los valores unicos de la columna
    np.unique(data_copia['room_type'])


    #Se reemplazan los datos categoricos a numerico
    data_copia['room_type'] = data_copia['room_type'].replace('Entire home/apt',1)
    data_copia['room_type'] = data_copia['room_type'].replace('Hotel room',2)
    data_copia['room_type'] = data_copia['room_type'].replace('Private room',3)
    data_copia['room_type'] = data_copia['room_type'].replace('Shared room',4)
    data_copia['room_type'] = data_copia['room_type'].astype(int)

    #Se eliminan columnas innecesarias
    data_evaluar = data_copia.select_dtypes(include=[int, float])
    data_evaluar.head(3)
    #Encontramos todos los coeficientes de correlacion entre las variables
    corr_factors = data_evaluar.corr()
    #Encontramos el valor absoluto de todas las correlaciones entre las variables
    corr_factors1 = abs(corr_factors)

    #Selecciono las columnas tipo numericas del dataframe Filtro_index1
    numeric_df1 = data_evaluar.select_dtypes(['float','int'])  #Devuelve Columnas
    numeric_cols1= numeric_df1.columns                         #Devuelve lista de Columnas

    numeric2_df1=data_evaluar[['host_response_rate','host_acceptance_rate','host_total_listings_count','accommodates','reviews_per_month','price']]


    return data_evaluar, numeric_df1, numeric_cols1, corr_factors, corr_factors1, numeric2_df1

#################################################################

#Cargo los datos obtenidos de la función "load_data"
data_evaluar, numeric_df1, numeric_cols1, corr_factors, corr_factors1, numeric2_df1= load_data()

######################CREACIÓN DEL DASHBOARD################################

#1. CREACIÓN DE LA SIDEBAR
#Generamos los encabezados para la barra lateral (sidebar)
st.sidebar.title("Airbnb RIO DE JANEIRO César Alejandro Rivera Guzmán")
st.sidebar.header("Actividad 3.6 Regresión NO Lineal")
st.sidebar.header('SIDEBAR')
st.sidebar.header("Panel de selección")



#Test 2 (streamlit run app.py)

#################################################################

#2. CREACIÓN DE LOS FRAMES
#Generamos los Frames que utilizaremos en el diseño
#Widget 1: Selectbox
#Menu desplegable de opciones de los Frames seleccionados
Frames= st.selectbox(label= "Frames", options= ["Descripción de la actividad",'Rio de Janeiro',"Regresiones"])

#Test 3 (streamlit run app.py)

############################# FRAME 1 ####################################

if Frames == 'Descripción de la actividad':
  st.title('Encontrar el mejor modelo de predicción para la variables estudio que son las siguientes:')
  st.write('Se evaluaron 11 modelos de regresion no lineal y se presentaran las variables estudio con sus variables independientes,\
  \n así como las predicciones que realizan entre otras variables')

######################### FRAME 2 ########################################
if Frames == "Rio de Janeiro":
    # Generamos los encabezados para el dashboard
    st.title("**Rio de Janeiro**")
    st.header("Conjunto de datos de la ciudad *Rio de Janeiro* **REGRESIÓN NO LINEAL**")
    st.subheader("Para esta sección serán las siguientes: : \
  \n-host_response_rate (variable objetivo)\
  \n-host_acceptance_rate (variable objetivo)\
  \n-host_total_listings_count (variable objetivo)\
  \n-accommodates (variable objetivo)\
  \n-reviews_per_month (variable objetivo)\
  \n-price(variable objetivo)')")

    #Widget 2: Checkbox
    check_box = st.sidebar.checkbox(label="Mostrar Mapa de Calor")
    if check_box:
      def HeatMap():
        plt.figure(figsize=(10, 8))  # Set the figure size
        Heat_Map = sns.heatmap(corr_factors1, cmap='Blues')
        plt.title('Mapa de calor Correlaciones Regresión NO lineal')
        st.pyplot(plt)  # Use st.pyplot to display the plot in Streamlit

      HeatMap()  # Call the HeatMap function
        #Mostramos el HEATMAP

    # Agregar la imagen RIO DE JANEIRO
    image_RIO_url = "https://media.architecturaldigest.com/photos/56f4781d904aa1ac0ac47256/16:9/w_2580,c_limit/Cristo%20Redentor.jpg"
    st.image(image_RIO_url, caption="Imagen de Rio de Janeiro", use_column_width=True)

    image_AIRBNB_url = 'https://brandemia.org/sites/default/files/sites/default/files/bnb_billboard_01-2000x1125.jpg'
    st.image(image_AIRBNB_url, caption="Airbnb", use_column_width=True)

    # Crear un mapa centrado en un lugar específico
    m = folium.Map(location=[-22.908333, -43.196388], zoom_start=5)

    # Agregar un marcador
    folium.Marker([-22.908333, -43.196388], popup="Rio de Janeiro, BRA").add_to(m)

    # Mostrar el mapa
    folium_static(m)

if Frames == "Regresiones":
    st.title("*Regresiones NO lineales*")
    st.write('Los **modelos** que mejores resultados fueron los siguientes:\
    \n-Modelo Exponencial\
    \n-Modelo Función Inversa\
    \n-Modelo Senoidal\
    \n-Modelo Tangencial\
    \n-Modelo Valor Absoluto\
    \n-Modelo Función Cociente entre Polinomios\
    \n-Modelo Logarítmica\
    \n-Modelo Función Lineal con Producto de Coeficientes\
    \n-Modelo Cuadrático Inverso\
    \nModelo Polinomial Inverso')

    # Generamos un cuadro de multi-selección (Y) para seleccionar variables a graficar
    Vars_num_selected = st.sidebar.multiselect(
        label="Variable dependiente de mayor coeficiente de correlación",
        options=[
            'host_response_rate',
            'host_acceptance_rate',
            'host_total_listings_count',
            'accommodates',
            'reviews_per_month',
            'price'
        ]
    )

    if Vars_num_selected:
        for var in Vars_num_selected:
            a = corr_factors1.sort_values(by=var, ascending=False).head(7)
            a = a[1:].head()
            st.write(f'Tabla Correlación para {var}')
            st.write(a)
            # Variable Independiente
            a1 = a.index[0]
            st.write('Variable dependiente:', a1)

            #Declaramos las variables dependientes e independientes para la regresión lineal
            vars_indep = data_evaluar[a1]
            var_dep = data_evaluar[var]
            x=vars_indep
            y=var_dep
                # Generamos un cuadro de multi-selección (Y) para seleccionar variables a graficar
            Vars_num_selected2 = st.sidebar.multiselect(
            label="Modelo de Regresión: ",
            options=[
            'Modelo Cuadrático',
            'Modelo Exponencial',
            'Modelo Función Inversa',
            'Modelo Senoidal',
            'Modelo Tangencial',
            'Modelo Valor Absoluto',
            'Modelo Función Cociente entre Polinomios',
            'Modelo Logarítmica',
            'Modelo Función Lineal con Producto de Coeficientes',
            'Modelo Cuadrático Inverso'
            'Modelo Polinomial Inverso'
        ]
    )
            if 'Modelo Cuadrático' in Vars_num_selected2:
              try:
                def func_Cuadratica(x, a, b, c):
                  return a * np.square(x) + b * x + c

                # Ajustamos los parámetros de la función curve_fit
                parametros, covs = curve_fit(func_Cuadratica, vars_indep, var_dep)

                # Predicciones
                y_pred = func_Cuadratica(vars_indep, *parametros)

                # Calculamos el coeficiente de determinación del modelo
                R2_Modelo14 = r2_score(var_dep, y_pred)
                st.write(f'Coeficiente de determinacion del modelo {var}:')
                st.write(R2_Modelo14)

                # Calculamos el coeficiente de correlación del modelo
                R_14 = np.sqrt(R2_Modelo14)
                st.write(f'Coeficiente de correlación del modelo {var}:')
                st.write(R_14)

                figure1 = px.scatter(data_frame=data_evaluar,
                        x=vars_indep,
                        y=[var_dep, y_pred],
                        title=str(var),
                        labels={'x': 'Valor de la variable independiente',
                                'y': 'Valor de la variable dependiente'},
                        width=1600, height=600,
                        color_discrete_sequence=['blue', 'red'],
                        color_discrete_map={'Valores reales': 'blue', 'Valores predecidos': 'red'})

                st.plotly_chart(figure1)
              except Exception as e:
                st.write(f"An unexpected error occurred: {e}")

            elif 'Modelo Exponencial' in Vars_num_selected2:
              try:
                def func_Expo(x, a, b, c):
                  return a * np.exp(-b * x) + c

                # Ajustamos los parámetros de la función curve_fit
                parametros, covs = curve_fit(func_Expo, vars_indep, var_dep)

                # Predicciones
                y_pred = func_Expo(vars_indep, *parametros)

                # Calculamos el coeficiente de determinación del modelo
                R2_Modelo14 = r2_score(var_dep, y_pred)
                st.write(f'Coeficiente de determinacion del modelo {var}:')
                st.write(R2_Modelo14)

                # Calculamos el coeficiente de correlación del modelo
                R_14 = np.sqrt(R2_Modelo14)
                st.write(f'Coeficiente de correlación del modelo {var}:')
                st.write(R_14)

                figure1 = px.scatter(data_frame=data_evaluar,
                        x=vars_indep,
                        y=[var_dep, y_pred],
                        title=str(var),
                        labels={'x': 'Valor de la variable independiente',
                                'y': 'Valor de la variable dependiente'},
                        width=1600, height=600,
                        color_discrete_sequence=['blue', 'red'],
                        color_discrete_map={'Valores reales': 'blue', 'Valores predecidos': 'red'})

                st.plotly_chart(figure1)
              except Exception as e:
                st.write(f"An unexpected error occurred: {e}")

            elif 'Modelo Función Inversa' in Vars_num_selected2:
              try:
                def func_inv(x, a, b, c):
                  return 1/a*x

                # Ajustamos los parámetros de la función curve_fit
                parametros, covs = curve_fit(func_inv, vars_indep, var_dep)

                # Predicciones
                y_pred = func_inv(vars_indep, *parametros)

                # Calculamos el coeficiente de determinación del modelo
                R2_Modelo14 = r2_score(var_dep, y_pred)
                st.write(f'Coeficiente de determinacion del modelo {var}:')
                st.write(R2_Modelo14)

                # Calculamos el coeficiente de correlación del modelo
                R_14 = np.sqrt(R2_Modelo14)
                st.write(f'Coeficiente de correlación del modelo {var}:')
                st.write(R_14)

                figure1 = px.scatter(data_frame=data_evaluar,
                        x=vars_indep,
                        y=[var_dep, y_pred],
                        title=str(var),
                        labels={'x': 'Valor de la variable independiente',
                                'y': 'Valor de la variable dependiente'},
                        width=1600, height=600,
                        color_discrete_sequence=['blue', 'red'],
                        color_discrete_map={'Valores reales': 'blue', 'Valores predecidos': 'red'})

                st.plotly_chart(figure1)
              except Exception as e:
                st.write(f"An unexpected error occurred: {e}")

            elif 'Modelo Senoidal' in Vars_num_selected2:
              try:
                def func_sen(x, a, b, c):
                  return a*np.sin(x) + b


                # Ajustamos los parámetros de la función curve_fit
                parametros, covs = curve_fit(func_sen, vars_indep, var_dep)

                # Predicciones
                y_pred = func_sen(vars_indep, *parametros)

                # Calculamos el coeficiente de determinación del modelo
                R2_Modelo14 = r2_score(var_dep, y_pred)
                st.write(f'Coeficiente de determinacion del modelo {var}:')
                st.write(R2_Modelo14)

                # Calculamos el coeficiente de correlación del modelo
                R_14 = np.sqrt(R2_Modelo14)
                st.write(f'Coeficiente de correlación del modelo {var}:')
                st.write(R_14)

                figure1 = px.scatter(data_frame=data_evaluar,
                        x=vars_indep,
                        y=[var_dep, y_pred],
                        title=str(var),
                        labels={'x': 'Valor de la variable independiente',
                                'y': 'Valor de la variable dependiente'},
                        width=1600, height=600,
                        color_discrete_sequence=['blue', 'red'],
                        color_discrete_map={'Valores reales': 'blue', 'Valores predecidos': 'red'})

                st.plotly_chart(figure1)
              except Exception as e:
                st.write(f"An unexpected error occurred: {e}")

            elif 'Modelo Tangencial' in Vars_num_selected2:
              try:
                def func_tan(x, a, b, c):
                  return a*np.tan(x) + b


                # Ajustamos los parámetros de la función curve_fit
                parametros, covs = curve_fit(func_tan, vars_indep, var_dep)

                # Predicciones
                y_pred = func_tan(vars_indep, *parametros)

                # Calculamos el coeficiente de determinación del modelo
                R2_Modelo14 = r2_score(var_dep, y_pred)
                st.write(f'Coeficiente de determinacion del modelo {var}:')
                st.write(R2_Modelo14)

                # Calculamos el coeficiente de correlación del modelo
                R_14 = np.sqrt(R2_Modelo14)
                st.write(f'Coeficiente de correlación del modelo {var}:')
                st.write(R_14)

                figure1 = px.scatter(data_frame=data_evaluar,
                        x=vars_indep,
                        y=[var_dep, y_pred],
                        title=str(var),
                        labels={'x': 'Valor de la variable independiente',
                                'y': 'Valor de la variable dependiente'},
                        width=1600, height=600,
                        color_discrete_sequence=['blue', 'red'],
                        color_discrete_map={'Valores reales': 'blue', 'Valores predecidos': 'red'})

                st.plotly_chart(figure1)
              except Exception as e:
                st.write(f"An unexpected error occurred: {e}")

            elif 'Modelo Valor Absoluto' in Vars_num_selected2:
              try:
                def func_abs(x, a, b, c):
                  return a*np.abs(x) + b*x + c

                # Ajustamos los parámetros de la función curve_fit
                parametros, covs = curve_fit(func_abs, vars_indep, var_dep)

                # Predicciones
                y_pred = func_abs(vars_indep, *parametros)

                # Calculamos el coeficiente de determinación del modelo
                R2_Modelo14 = r2_score(var_dep, y_pred)
                st.write(f'Coeficiente de determinacion del modelo {var}:')
                st.write(R2_Modelo14)

                # Calculamos el coeficiente de correlación del modelo
                R_14 = np.sqrt(R2_Modelo14)
                st.write(f'Coeficiente de correlación del modelo {var}:')
                st.write(R_14)

                figure1 = px.scatter(data_frame=data_evaluar,
                        x=vars_indep,
                        y=[var_dep, y_pred],
                        title=str(var),
                        labels={'x': 'Valor de la variable independiente',
                                'y': 'Valor de la variable dependiente'},
                        width=1600, height=600,
                        color_discrete_sequence=['blue', 'red'],
                        color_discrete_map={'Valores reales': 'blue', 'Valores predecidos': 'red'})

                st.plotly_chart(figure1)
              except Exception as e:
                st.write(f"An unexpected error occurred: {e}")

            elif 'Modelo Función Cociente entre Polinomios' in Vars_num_selected2:
              try:
                def func_coci_poli(x, a, b, c):
                  return (a*x**2 + b)/ c*x

                # Ajustamos los parámetros de la función curve_fit
                parametros, covs = curve_fit(func_coci_poli, vars_indep, var_dep)

                # Predicciones
                y_pred = func_coci_poli(vars_indep, *parametros)

                # Calculamos el coeficiente de determinación del modelo
                R2_Modelo14 = r2_score(var_dep, y_pred)
                st.write(f'Coeficiente de determinacion del modelo {var}:')
                st.write(R2_Modelo14)

                # Calculamos el coeficiente de correlación del modelo
                R_14 = np.sqrt(R2_Modelo14)
                st.write(f'Coeficiente de correlación del modelo {var}:')
                st.write(R_14)

                figure1 = px.scatter(data_frame=data_evaluar,
                        x=vars_indep,
                        y=[var_dep, y_pred],
                        title=str(var),
                        labels={'x': 'Valor de la variable independiente',
                                'y': 'Valor de la variable dependiente'},
                        width=1600, height=600,
                        color_discrete_sequence=['blue', 'red'],
                        color_discrete_map={'Valores reales': 'blue', 'Valores predecidos': 'red'})

                st.plotly_chart(figure1)
              except Exception as e:
                st.write(f"An unexpected error occurred: {e}")

            elif 'Modelo Logarítmica' in Vars_num_selected2:
              try:
                def func_log(x, a, b, c):
                  return a*np.log(x) + b

                # Ajustamos los parámetros de la función curve_fit
                parametros, covs = curve_fit(func_log, vars_indep, var_dep)

                # Predicciones
                y_pred = func_log(vars_indep, *parametros)

                # Calculamos el coeficiente de determinación del modelo
                R2_Modelo14 = r2_score(var_dep, y_pred)
                st.write(f'Coeficiente de determinacion del modelo {var}:')
                st.write(R2_Modelo14)

                # Calculamos el coeficiente de correlación del modelo
                R_14 = np.sqrt(R2_Modelo14)
                st.write(f'Coeficiente de correlación del modelo {var}:')
                st.write(R_14)

                figure1 = px.scatter(data_frame=data_evaluar,
                        x=vars_indep,
                        y=[var_dep, y_pred],
                        title=str(var),
                        labels={'x': 'Valor de la variable independiente',
                                'y': 'Valor de la variable dependiente'},
                        width=1600, height=600,
                        color_discrete_sequence=['blue', 'red'],
                        color_discrete_map={'Valores reales': 'blue', 'Valores predecidos': 'red'})

                st.plotly_chart(figure1)
              except Exception as e:
                st.write(f"An unexpected error occurred: {e}")

            elif 'Modelo Función Lineal con Producto de Coeficientes' in Vars_num_selected2:
              try:
                def func_lin_prod(x, a, b, c):
                  return a*x + b*x + c*x

                # Ajustamos los parámetros de la función curve_fit
                parametros, covs = curve_fit(func_lin_prod, vars_indep, var_dep)

                # Predicciones
                y_pred = func_lin_prod(vars_indep, *parametros)

                # Calculamos el coeficiente de determinación del modelo
                R2_Modelo14 = r2_score(var_dep, y_pred)
                st.write(f'Coeficiente de determinacion del modelo {var}:')
                st.write(R2_Modelo14)

                # Calculamos el coeficiente de correlación del modelo
                R_14 = np.sqrt(R2_Modelo14)
                st.write(f'Coeficiente de correlación del modelo {var}:')
                st.write(R_14)

                figure1 = px.scatter(data_frame=data_evaluar,
                        x=vars_indep,
                        y=[var_dep, y_pred],
                        title=str(var),
                        labels={'x': 'Valor de la variable independiente',
                                'y': 'Valor de la variable dependiente'},
                        width=1600, height=600,
                        color_discrete_sequence=['blue', 'red'],
                        color_discrete_map={'Valores reales': 'blue', 'Valores predecidos': 'red'})

                st.plotly_chart(figure1)
              except Exception as e:
                st.write(f"An unexpected error occurred: {e}")

            elif 'Modelo Cuadrático Inverso' in Vars_num_selected2:
              try:
                def func_cuad_inv(x, a, b, c):
                  return 1/a*x**2

                # Ajustamos los parámetros de la función curve_fit
                parametros, covs = curve_fit(func_cuad_inv, vars_indep, var_dep)

                # Predicciones
                y_pred = func_cuad_inv(vars_indep, *parametros)

                # Calculamos el coeficiente de determinación del modelo
                R2_Modelo14 = r2_score(var_dep, y_pred)
                st.write(f'Coeficiente de determinacion del modelo {var}:')
                st.write(R2_Modelo14)

                # Calculamos el coeficiente de correlación del modelo
                R_14 = np.sqrt(R2_Modelo14)
                st.write(f'Coeficiente de correlación del modelo {var}:')
                st.write(R_14)

                figure1 = px.scatter(data_frame=data_evaluar,
                        x=vars_indep,
                        y=[var_dep, y_pred],
                        title=str(var),
                        labels={'x': 'Valor de la variable independiente',
                                'y': 'Valor de la variable dependiente'},
                        width=1600, height=600,
                        color_discrete_sequence=['blue', 'red'],
                        color_discrete_map={'Valores reales': 'blue', 'Valores predecidos': 'red'})

                st.plotly_chart(figure1)
              except Exception as e:
                st.write(f"An unexpected error occurred: {e}")

            elif 'Modelo Polinomial Inverso' in Vars_num_selected2:
              try:
                def func_poli_inv(x, a, b, c):
                  return a/b*x**2 + c*x

                # Ajustamos los parámetros de la función curve_fit
                parametros, covs = curve_fit(func_poli_inv, vars_indep, var_dep)

                # Predicciones
                y_pred = func_poli_inv(vars_indep, *parametros)

                # Calculamos el coeficiente de determinación del modelo
                R2_Modelo14 = r2_score(var_dep, y_pred)
                st.write(f'Coeficiente de determinacion del modelo {var}:')
                st.write(R2_Modelo14)

                # Calculamos el coeficiente de correlación del modelo
                R_14 = np.sqrt(R2_Modelo14)
                st.write(f'Coeficiente de correlación del modelo {var}:')
                st.write(R_14)

                figure1 = px.scatter(data_frame=data_evaluar,
                        x=vars_indep,
                        y=[var_dep, y_pred],
                        title=str(var),
                        labels={'x': 'Valor de la variable independiente',
                                'y': 'Valor de la variable dependiente'},
                        width=1600, height=600,
                        color_discrete_sequence=['blue', 'red'],
                        color_discrete_map={'Valores reales': 'blue', 'Valores predecidos': 'red'})

                st.plotly_chart(figure1)
              except Exception as e:
                st.write(f"An unexpected error occurred: {e}")




##############################################
#Test 4 (streamlit run app.py)


Overwriting app.py


# Link Tunnel Dashboard y contraseña

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.74.76.202



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.74.76.202:8501

your url is: https://sixty-lines-beg.loca.lt
/content/app.py:145: DeprecationWarning: 
folium_static is deprecated and will be removed in a future release, or
simply replaced with with st_folium which always passes
returned_objects=[] to the component.
Please try using st_folium instead, and
post an issue at https://github.com/randyzwitch/streamlit-folium/issues
if you experience issues with st_folium.

  folium_static(m)
/content/app.py:145: DeprecationWarning: 
folium_static is deprecated and will be removed in a future release, or
simply replaced with with st_folium which always passes
returned_objects=[] to the component.
Please try using st_folium instead, and
post an issue at https://github.com/randyzwitch/streamlit-folium/issues
if you experience issues with st_folium.

  folium_static(m)
/cont